Frederick Lancia
Eutopia Task 2
April 28

In [5]:
import re
from bs4 import BeautifulSoup as bs
from pip._vendor import requests
import pandas as pd
from googletrans import Translator

# This class will try multiple regex patterns on the same text
# until it finds a date or fails all of them
# Disclaimer: this will not help you find a date, try Tinder!
class date_finder:
    def find(self):
        # now look for a where that month appears in the text

        text = self.text
        month = self.month
    
        # if I had a little more time, I would probably have automated
        # this better with another function to be called 4 times
        
        # month day, year
        attempt2 = re.findall(month+'.? \d{1,2}, [1-2]\d{3}', text, 
                    re.IGNORECASE)
        if attempt2:
            return attempt2[0]

        # day month year
        attempt3 = re.findall('\d{1,2} '+ month +'.? [1-2]\d{3}', text, 
                    re.IGNORECASE)
        if attempt3:
            return attempt3[0]
        
        # month year
        attempt4 = re.findall(month+'.? [1-2]\d{3}', text, re.IGNORECASE)
        if attempt4:
            return attempt4[0]

        # no success :/
        return ''

    def __init__(self, text, month):
        self.text = text
        self.month = month

# currency finder class takes a string of text and a string (word or symbol)
# indicating currency and returns a list containing a float representing
# the amount of money in its original currency that was invested
# disclaimer: neither will this class make you rich, try a startup!
class currency_finder:
    def find(self):
        # ok heres the plan. look at each paragraph in order
        # first look for the simple date in numbers
        # next look for a month
        # if you find a month, record it, and now look for a pattern
        # with that month inside

        text = self.text
        currency = self.currency

        # Note: if I had a little more time, I would probably have automated
        # this better with another function to be called 4 times

        for figure in figures:
            figure_found = re.findall(figure, text, re.IGNORECASE)
            if figure_found:
                #you found a figure! Now you have everything you need to find
                # a full 'money phrase'
                if currency in symbols:
                    # symbols usually come before the money amount as in the 
                    # following format:
                    # symbol number figure: $5.5 thousand
                    # to be sure that the $ sign does not cause any problems, 
                    # the dashes mitigate the errors
                    if currency == '$': currency = "\\" + currency
                    # look for a phrase in the above format
                    phrase_found = re.findall(currency+" ?\d+[.]?\d* ?"+figure, 
                                    text, re.IGNORECASE)
                    if phrase_found:
                        # if found, pull the number out from it
                        number_pattern = r'\d+[.]?\d*'
                        num = re.findall(number_pattern, phrase_found[0])
                        # use the number and figure to calculate the amount 
                        # invested.
                        # return a list containing a float
                        return self.calculator(num[0], figure)
                else:
                    # you found a currency
                    # currencies usually come after the figure

                    # number figure currency 5.5 thousand dollars
                    phrase_found = re.findall('\d+[.]?\d*? ?'+figure + currency,
                                     text, re.IGNORECASE)
                    if phrase_found:
                        number_pattern = r'\d+[.]?\d*'
                        num = re.findall(number_pattern, phrase_found[0])
                        return self.calculator(num[0], figure)

        # no success :/
        return []
    
    # calculator takes the number and figure (thousand, million, etc) and 
    # returns a long float
    def calculator(self, num, figure):
        if figure == 'thousand':
            final_amount = [float(float(num)*1000)]
        elif figure == 'billion':
            final_amount = [float(float(num)*1000000000)]
        else:
            # if in another language, assume million. It is the most common
            final_amount = [float(float(num)*1000000)]
        return final_amount


    def __init__(self, text, currency):
        self.text = text
        self.currency = currency
    
# translate_lists is a class that takes a list of strings and appends 
# translated versions of each string (in the language of the current site) 
# and returns them as a larger list
class translate_list:
    def trans(self):
        words = self.list.copy()
        # for every word in list, add its translation in this language and 
        # return longer list
        for word in self.list:
            translated_word = translator.translate(word, dest=language, 
                                src='en')
            words.add(translated_word.text)
        return words

    def __init__(self, list):
        self.list = list

# pull key currency related values from website database to help
# identify money phrases
currencies_link = 'https://wise.com/gb/blog/world-currency-symbols'

# use sets to record key currency related values, because we don't need 
# repeats while we will look for codes and currencies in the same places, 
# we will keep them separated for now, so that we can translate currencies 
# later without translating codes
countries = set([])
currencies = set([])
codes = set([])
symbols = set([])
iteration = 0

# set up html parser
currencies_request = requests.get(currencies_link)
currencies_content = currencies_request.content
currencies_soup = bs(currencies_content)
tbodies = currencies_soup.find_all('tbody')
# get text from table bodies
for tbody in tbodies:

    individual_currencies = tbody.find_all(["tr"])
    for row in individual_currencies:
        # once converted into text, values are separated by '  '
        # we want to break values back into their parts
        # the first 4 tables contain country names as their
        # first value. After the 4th table, they provide only
        # the last 3 value types
        parts = row.text.split('  ')
        
        if iteration < 4:
            countries.add(parts[0])

            stripped_currency = parts[1].strip()
            currencies.add(stripped_currency)
            # add last word of each currency, as some
            # will be written like this
            # for example, dollar instead of US dollar
            currencies.add(stripped_currency.split()[-1])
            codes.add(parts[2])
            symbols.add(parts[3])
        else :
            currencies.add(parts[0].strip())
            codes.add(parts[1])
            symbols.add(parts[2])
    iteration+=1

# the following values cause technical problems, so remove them
currencies.remove("South african rand**")
currencies.remove("rand**")
symbols.remove("R")

# $ causes problems on its own
symbols.add('\$')

xls = pd.ExcelFile(r'InputData.xlsx')
fround_data = pd.read_excel(xls, 'Fround')

# initialize translator element
translator = Translator()

# initialize data frame to fill with extracted data and return as xls
new_df = fround_data.copy()
new_df["amount_of_round"] = "N/A"
new_df["article_publication_date"] = "N/A"

# set of all the months for finding dates
months = set(["January", "February", "March", "April", "May", "June", 
            "July", "August", "September", "October", "November", 
            "December", "Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul",
            "Aug", "Sep", "Oct", "Nov", "Dec"])

# set for finding english indicators of big numbers which are often used to 
# describe amounts of currency
figures = set(["thousand", "million", "billion"])

# use past translations, because the translator is extremely slow and tends 
# to send lost connection or timeout errors
language_words2_df = pd.read_pickle('language_words2.pkl')
"""""
uncomment this code and delete this text if you want to run the program without 
the language file. You will also need to comment out the above line of code

english_list = [months, figures, currencies]
starter_data = [['en', english_list]]
language_words2_df = pd.DataFrame(starter_data, columns = ['language', 'words'])
"""


# iterate through every row of given xls file containing article information
for i, row in fround_data.iterrows():
    
    article_url = row['news_url'] # extract article url

    company_name = row['name'] # extract company name
    
    # create html parser
    article_request = requests.get(article_url)
    article_content = article_request.content
    article_soup = bs(article_content)
    
    # split visual text into paragraphs, because some bits of text which
    # should read together are in different elements
    # use first paragraph to detect language of website
    # assume that language remains constant throughout site
    paragraphs = article_soup.findAll(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6'])
    paragraphs_text = []
    for paragraph in paragraphs:
        paragraph_text = paragraph.text
        paragraphs_text.append(paragraph.text)

    # nearly all articles will have some paragraphs, if not, assume English
    if paragraphs_text: language = translator.translate(paragraphs_text[0]).src
    else: language = 'en'

    # ask if this language has already been translated
    if language not in set(language_words2_df['language']):
        # data for this language doesn't yet exist in the data frame
        # we need to translate our words to this language and add it to df

        new_language_list = [translate_list(months).trans(), 
                            translate_list(figures).trans(),
                            translate_list(currencies).trans()]
        new_row_data = pd.DataFrame([[language, new_language_list]],
                            columns=['language', 'words'])
        language_words2_df = pd.concat([language_words2_df, new_row_data], 
                        ignore_index=True)
        # isolate the row of the language this article is in into it's own 
        # dataframe
    language_row = language_words2_df.loc[language_words2_df['language'] == 
                        language]

    # extract key words from stored words and their translation in site's 
    # language
    months = language_row.iat[0,1][0]
    figures = language_row.iat[0,1][1]
    currencies = language_row.iat[0,1][2]

    # create list to contain the article publish date
    date = []

    # ok heres the plan. look at each paragraph in order
    # first look for the simple date in numbers
    # next look for a month
    # if you find a month, record it, and now look for a pattern
    # with that month inside

    # if html contains element "time", set this text equal to date
    times = article_soup.find_all(["time"])
    if times:
        date = times[0].text
    else:
        # if contains element post-date-default, set this text equal to date
        times = article_soup.select("div#post-date-default")
        if times:
            date = times[0].text
        else: 
            # if contains element entry-content, check to see if it contains a 
            # date
            times = article_soup.select("div#entry-content")
            if times:
                for time in times:
                    text = time.text

                    simple_date_pattern = r'd{2}[-/ ]+\d{2}[-/ ]+/d{2,4}'
                    attempt1 = re.findall(simple_date_pattern, text, re.IGNORECASE)
                    if attempt1:
                        date = attempt1[0]
                        break
                    # if it contains a date, set that text equal to date
                    for month in months:
                        text = time.text
                        month_found = re.findall(month, text, re.IGNORECASE)

                        if month_found:
                            date = date_finder(text, month_found[0]).find()
                            if date: break
                    if date: break

    # if all of those methods failed to find a date...
    if not date:
        # now look at the general text from start to finish by paragraph
        for text in paragraphs_text:
            # first try the simplest form of a date in numbers
            # dd/- dd/- year
            simple_date_pattern = r'd{2}[-/ ]+\d{2}[-/ ]+/d{2,4}'
            attempt1 = re.findall(simple_date_pattern, text, re.IGNORECASE)
            if attempt1:
                date = attempt1[0]
                break

            # search for each month in this paragraph
            for month in months:
                month_found = re.findall(month, text, re.IGNORECASE)
                # if month is found, there's a good chance its part of a date!
                if month_found:
                    # try to find the date
                    date = date_finder(text, month_found[0]).find()
                    if date: break
                if date: break
    # find amount of money in original currency
    # they can be in the form of '$ 5 million' or '5.5 million SEK'.

    # now that currencies have been translated if needed, combine currencies and 
    # codes
    currencies |= codes

    # the money sign continues to be problematic, this fixes it
    currencies.add('\$')

    # create a set for searching through all important indicators at once
    currencies_union = set.union(currencies, symbols)

    # iterate through text in paragraphs
    for text in paragraphs_text:
        # search for every indicator in the paragraph
        for currency in currencies_union:
            currency_found = re.findall(currency, text, re.IGNORECASE)

            # if found, see if it is part of what we are looking for
            if currency_found:
                amount = currency_finder(text, currency_found[0]).find()
                # break if found
                if amount:break
        if amount:break

    # if we never find the money ammount, set equal to float 0.0 for lack of better 
    # float
    if not amount: amount = [0.0]

    # found values to new_df
    new_df.at[i, "amount_of_round"] = amount[0]
    new_df.at[i, "article_publication_date"] = date

    # save dictionary of translated words dataframe to pickle for further use
    language_words2_df.to_pickle(r'language_words2.pkl')

# save created dataframe to excel file
new_df.to_excel(r'LanciaEutopiaTask2.xlsx', index = False, header=True)

ValueError: No engine for filetype: 'ipynb'